<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Chapter08: Inversion example
***

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In geophysics, our *measurements* are often *indirect* measurements of a material property characterising the sub-surface. Examples:

- **Gravity:** The Bouguer anomaly $\Delta g_b$ [mGal] maps density differences $\Delta \rho$ [kg/m$^3$] in the sub-surface.

- **Magnetics:** The Total-field anomaly $T$ [T] maps the presense of magnetised material $M$ [A/m].

- **Geoelectrics:** The measured potential difference $U$ [V] maps the resistivity $\rho$ [$\Omega$m] of the sub-surface.

We formalise the above-mentioned statements with the definition of data and model.

We define a *data vector* as
$$
d_i = (d_1,d_2, \dots, d_n)^T, i=1,n
$$
with $n$ measured data points.

We define a model, which predicts these data points for the same locations as *prediction vector*
$$
p_i = (p_1,p_2,\dots,p_n)^T, i=1,n
$$

The underlying model used for the prediction is based on our *physical knowledge* of the sub-surface. E.g. for predicting the Bouguer gravity, we define a (three-dimensional) density distribution of the sub-surface. By adjusting the different density values, we try to match data and model vectors:
$$
d_i \simeq p_i
$$

## Error estimation

If we want to express the above relation between data and model vector as accurate, we have to allow for an
*error vector* $e_i$, describing the deviation between data and model at a particular point:
$$
d_i = p_i + e_i \Delta d_i
$$
Here, $\Delta d_i$ is the data *uncertainty*.

Solving the equation for the error vector, we obtain:
$$
e_i = {{d_i - p_i} \over {\Delta d_i}}
$$

Instead of looking at each single error for each location, we summarize all errors with the $\chi^2$-function
as a *goodness-of-fit*:
$$
\chi^2 = \sum\limits_{i=1}^{n} \left( {{d_i - p_i} \over {\Delta d_i}} \right)^2
$$

We identify the $\chi^2$-function as the scalar product of the error vector by rewriting:
$$
\chi^2 
=\sum\limits_{i=1}^{n} \left( {{d_i - p_i} \over {\Delta d_i}} \right)^2
=[d_i - p_i]^T \left( {{1} \over {\Delta d_i}} \right)^2 [d_i - p_i]
= [d_i - p_i]^T 
\left( \begin{array}{ccc}
{{1}\over{\Delta d_1^2}} & 0 & 0 \\
0&\dots&0\\
0&0&{{1}\over{\Delta d_1^2}}
\end{array} \right) [d_j - p_j]
= [d_i - p_i]^T [Co^d_{ij}]^{-1} [d_j - p_j]
$$
Note that we are picky in writing the error vector once as column, once as row vector (transposed). Also, the
data uncertainties have been expanded into a matrix, with the uncertainties $\Delta d_i$ on the main diagonal.

We call $[Co^d_{ij}]$ the *co-variance matrix*.

We will see later, why we complicated the goddness-of-fit...

## Non-linear models

Next, we need to define our *preditive model* $G$. 
As a general setup, we use a prediction $G$ based on a pre-defined start model vector $m$:
$$
p_i = G_{ij} (m_j)
$$
Here, our knowledge about the processes is described as model matrix $G_{ij}$ with non-linear model coefficients
$m_i$. The model is called *non-linear inverse problem*.

An example: We have two Sinusoidal time series with different amplitudes $A_i$ and different 
frequencies $\omega_i$:
$$
 p_i = A_1 \sin (\omega_1 t) + A_2 \sin (\omega_2 t)
$$
While this model is *linear* in the coefficients $A_1$ and $A_2$, it is *non-linear* with respect to the
two frequenies, $\omega_1$ and $\omega_2$. Our model vector is $m_j=(A_1,A_2,\omega_1,\omega_2)^T$, and as
data vector we would take some samples from the time series, $d_i=d_i(t_i)$.

## Linear models

We start discussing a linear problem first. As an example, assume a straight line fit:
$$
p_i = a + b x_i
$$

In that case we can separate the model vector $m_j$ from the predictive model $G_{ij}$:
$$
p_i = G_{ij} m_j
$$
Now, the matrix is decoupled from the model vector, and the problem can be called *linear inverse problem*.

With this definition, we can rewrite the goodness-of-fit to:

$$
\chi^2 
=\sum\limits_{i=1}^{n} \left( {{d_i - G_{ij}m_j} \over {\Delta d_i}} \right)^2
=[d_i - G_{ij}m_j]^T \left( {{1} \over {\Delta d_i}} \right)^2 [d_i - G_{ij}m_j]
= [d_i - G_{ij} m_j]^T 
\left( \begin{array}{ccc}
{{1}\over{\Delta d_1^2}} & 0 & 0 \\
0&\dots&0\\
0&0&{{1}\over{\Delta d_1^2}}
\end{array} \right) [d_i - G_{ij} m_j]
= [d_i - G_{ij} m_j]^T [Co^d]^{-1} [d_i - G_{ij} m_j]
$$

A solution of the linear inverse problem is now found by minimising the least-squares term $\chi^2$. This is
achieved by taking the derivatives of $\chi^2$ with respect to the model vector $m_j$ and setting each
derivative to zero, because this marks our extremum. We will demonstrate this with an example.

## Example: Linear Regression

We define a set of points distributed irregularly along the $x$-axis, $d_i=d_(x_i)$:

In [ ]:
# define data vector
n = 101
x = np.sort(np.random.uniform(-2,2,n))
#x = np.linspace(-2,2,n)
d = x**3 + np.random.uniform(-2,2,n)

# plot data points
plt.plot(x,d,marker='.',linestyle='none',color='red',label='data')
plt.legend()
plt.show()

## Polynomial fit

We can now make use of the in-build *polyfit*-function of python (numpy) and fit a linear polynom through the data points:

In [ ]:
plt.style.use('ggplot')
# function to reassemble polynomial
def poly(x,coeff):
    n = len(coeff)-1
    y = 0
    for i in range(0,len(coeff)):
        y = y + coeff[i]*x**(n-i)
    return y

# define data vector
n = 101
x = np.sort(np.random.uniform(-2,2,n))
#x = np.linspace(-2,2,n)
d = x**3 + np.random.uniform(-2,2,n)

# calculate fitting polynomial of degree m
m = 3
coeff = np.polyfit(x,d,m)
print ('coeff: ',coeff)

# calculate prediction
p=poly(x,coeff)

# plot data and model vector
plt.plot(x,d,marker='.',linestyle='none',color='red',label='data')
plt.plot(x,p,linestyle='-',color='blue',label='polynom of degree '+str(m))
plt.grid(True)
plt.legend()
plt.show()

### Analytical linear fit

However, we want to know, how the fitting procedure works.

We first derive an analytical solution for the least-squares fit using a linear regression function:
$$
p_i = a + b x_i
$$

Inserting this function into our goodness-of-fit criterion:
$$
\chi^2 = \sum\limits_{i=1}^{n} \left( {{d_i - p_i} \over {\Delta d_i}} \right)^2
= \sum\limits_{i=1}^{n} \left( {{d_i - a - b x_i} \over {\Delta d_i}} \right)^2
$$

We take the derivative of $\chi^2$ with respect to the model coefficients $a$ and $b$:
$$
\begin{array}{rclcl}
{{\partial \chi^2}\over{\partial a}} 
& = & \sum\limits_{i=1}^{n} 2 \left( {{d_i - a - b x_i} \over {\Delta d_i^2}} \right) (-1) &=&0\\
{{\partial \chi^2}\over{\partial b}} 
& = & \sum\limits_{i=1}^{n} 2 \left( {{d_i - a - b x_i} \over {\Delta d_i^2}} \right) (-x_i) &=&0
\end{array}
$$

Rearranging these two equations leads to the linear system of equations:
$$
\begin{array}{rcl}
 \sum\limits_{i=1}^{n} {{a} \over {\Delta d_i^2}}
+\sum\limits_{i=1}^{n} {{b x_i} \over {\Delta d_i^2}}
&=&\sum\limits_{i=1}^{n} {{d_i} \over {\Delta d_i^2}} \\
 \sum\limits_{i=1}^{n} {{a x_i} \over {\Delta d_i^2}}
+\sum\limits_{i=1}^{n} {{b x_i^2} \over {\Delta d_i^2}}
&=&\sum\limits_{i=1}^{n} {{d_i x_i} \over {\Delta d_i^2}}
\end{array}
$$
or, by moving the coefficients $a$ and $b$ in front of the sums:
$$
\begin{array}{rcl}
 a \sum\limits_{i=1}^{n} {{1} \over {\Delta d_i^2}}
+b \sum\limits_{i=1}^{n} {{x_i} \over {\Delta d_i^2}}
&=&\sum\limits_{i=1}^{n} {{d_i} \over {\Delta d_i^2}} \\
 a \sum\limits_{i=1}^{n} {{x_i} \over {\Delta d_i^2}}
+b \sum\limits_{i=1}^{n} {{x_i^2} \over {\Delta d_i^2}}
&=&\sum\limits_{i=1}^{n} {{d_i x_i} \over {\Delta d_i^2}}
\end{array}
$$

We can rearrange these to equations to formal $2x2$-system of euqations
$$
\left(
\begin{array}{cc}
\sum\limits_{i=1}^{n} {{1} \over {\Delta d_i^2}} & \sum\limits_{i=1}^{n} {{x_i} \over {\Delta d_i^2}} \\
\sum\limits_{i=1}^{n} {{x_i} \over {\Delta d_i^2}} & \sum\limits_{i=1}^{n} {{x_i^2} \over {\Delta d_i^2}}
\end{array}
\right)
\left(
\begin{array}{rcl} a \\ b
\end{array}
\right)
= \left(
\begin{array}{cc}
\sum\limits_{i=1}^{n} {{d_i} \over {\Delta d_i^2}} \\
\sum\limits_{i=1}^{n} {{d_i x_i} \over {\Delta d_i^2}}
\end{array}
\right)
$$ 
for the two unknowns $a$ and $b$, which has a unique solution.

In [ ]:
# define data vector
n    = 101
x    = np.sort(np.random.uniform(-2,2,n))
d    = x**3 + np.random.uniform(-2,2,n)
derr = np.ones(n)*2.

# solve linear regression
A11 = sum(1/derr**2)
A12 = sum(x/derr**2)
A21 = A12
A22 = sum(x**2/derr**2)
B1  = sum(d/derr**2)
B2  = sum(x*d/derr**2)
delta = A11*A22 - A12**2
a = (A22*B1-A12*B2) / delta
b = (A11*B2-A12*B1) / delta
print (a,b)

# plot data and model vector
plt.plot(x,d,marker='.',linestyle='none',color='red',label='data')
plt.plot(x,a+b*x,linestyle='-',color='blue',label='linear fit with y-errors')
plt.grid(True)
plt.legend()
plt.show()

### Formal linear inversion

Remember that we defined our *linear inverse problem* as:
$$
p_i = a + b x_i
$$
and the *goodness-of-fit* as:
$$
\chi^2 = [d_i - p_i]^T [Co^d_{ij}]^{-1} [d_j - p_j]
$$
with $d_i$ the data vector.

Inserting the linear inverse problem into the goodness-of-fit then yields:
$$
\chi^2 = [d_i - G_{ij} m_j]^T [Co^d]^{-1} [d_i - G_{ij} m_j]
$$
where our **model vector** has been defined as
$$
m_i = (a,b)^T
$$
and the **model matrix** as
$$
G_{ij} = 
\left( \begin{array}{cc}
1 & x_1 \\
\dots & \dots \\
1 & x_n
\end{array} \right)
$$
and the data **covariance matrix**:
$$
Co^d_{ij} =
\left( \begin{array}{ccc}
{{1}\over{\Delta d_1^2}} & 0 & 0 \\
0&\dots&0\\
0&0&{{1}\over{\Delta d_1^2}}
\end{array} \right)
$$

The solution vector $m_i$ is found by taking the partial derviatives from $\chi^2$ with respect to $m_j$:
$$
m_j = \left\{ G_{ij}^T (Co^d_{ii})^{-1} G_{ij} \right\}^{-1} G_{ij}^T (Co^d_{ii})^{-1} d_i
$$

We therefore need to assemble all matrices, vectors, calculate inverse and transpose matrices,
and combine them ...

In [ ]:
from numpy import matmul
from numpy.linalg import inv as matinv
# define data vector
n    = 101
x    = np.sort(np.random.uniform(-2,2,n))
d    = x**3 + np.random.uniform(-2,2,n)
derr = np.ones(n)*2.

# define data covariance matrix and its inverse
Cod  = np.eye(n,n)
for i in np.arange(0,len(derr)):
    Cod[i][i] = 1. / derr[i]**2
Codi = matinv(Cod)

# define model matrix
G    = np.array([[1,x[0]]])
for i in np.arange(1,len(d)):
    G = np.append(G,[[1,x[i]]],axis=0)

# calculate model vector
GT_Codi_Gi = matinv(matmul(G.transpose(),matmul(Codi,G)))
GT_Codi_d  = matmul(G.transpose(),matmul(Codi,d))
model      = matmul(GT_Codi_Gi,GT_Codi_d)
print (model.ndim,model.shape,model)

# plot data and model vector
plt.plot(x,d,marker='.',linestyle='none',color='red',label='data')
plt.plot(x,model[0]+model[1]*x,linestyle='-',color='blue',label='formal linear inversion')
plt.grid(True)
plt.legend()
plt.show()

--- continue ---

In [ ]:
import numpy as np
a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
b1 = np.array([10,11])
#b = b[np.newaxis,:]
print ('original')
print (a1.shape,'\n',a1)
print (b1.shape,'\n',b1)
# transpose
a2 = a1.transpose()
b2 = b1.transpose()
print ('transposed')
print (a2.shape,'\n',a2)
print (b2.shape,'\n',b2)
# inverted
print ('inverted')
a1 = np.array([[1,2,3],[0,1,5],[5,6,0]])
a3 = np.linalg.inv(a1)
print(a1)
print (a3.shape,'\n',a3)
np.allclose(np.dot(a1, a3), np.eye(a1.shape[0]))

In [ ]:
import numpy as np
a = np.array([1,2,3,4])
print ('a: ',a.shape,'\n',a)
b=a[np.newaxis,:]
print ('b: ',b.shape,'\n',b)
c=a[:,np.newaxis,]
print ('c: ',c.shape,'\n',c)
print (c.transpose())

In [ ]:
import numpy as np
print(np.array([[1],[2],[3]]))
print(np.array([[1,2,3]]))